In [2]:
!pip install mecab-python3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 23.9 MB/s eta 0:00:00


In [5]:
import os
import json
import torch
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch.utils.data import Dataset, DataLoader
from concurrent.futures import ThreadPoolExecutor 
import gradio as gr
import zipfile
import pickle
import MeCab

In [ ]:

# 1. Mecab 형태소 분석기 초기화
mecab = MeCab.Tagger()

# 2. Mecab을 이용한 형태소 분석 및 전처리 함수
def preprocess_with_mecab(text):
    """Mecab을 사용하여 텍스트를 전처리하는 함수"""
    words = mecab.parse(text).split()  # Mecab 사용 시
    return " ".join(words)

# 3. JSON 파일을 로드하고 형태소 분석을 적용하는 함수
def load_single_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            question = data.get('question', {}).get('comment')  # 질문 필드
            context = data.get('context')  # 문맥 필드
            answer = data.get('answer', {}).get('comment')  # 답변 필드

            # Mecab으로 전처리
            if question:
                question = preprocess_with_mecab(question)
            if context:
                context = preprocess_with_mecab(context)
            if answer:
                answer = preprocess_with_mecab(answer)

            # answer_start는 문맥 내에서 답변 시작 위치를 찾음
            answer_start = context.find(answer) if answer in context else -1

            if question and context and answer and answer_start != -1:
                return (question, context, answer, answer_start)
        except json.JSONDecodeError as e:
            print(f"JSON 디코딩 오류: {e}, 파일: {file_path}")
    return None

# 4. 캐시 관련 함수
def load_data_from_cache(cache_file):
    """캐시된 데이터를 불러오는 함수"""
    if os.path.exists(cache_file):
        print(f"캐시된 데이터를 불러오는 중: {cache_file}")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    return None

def save_data_to_cache(data, cache_file):
    """데이터를 캐시에 저장하는 함수"""
    with open(cache_file, 'wb') as f:
        pickle.dump(data, f)
    print(f"데이터를 캐시에 저장했습니다: {cache_file}")

# 5. 병렬로 데이터를 로드하는 함수
def load_data_parallel(data_folder):
    qa_pairs = []
    print(f"데이터 폴더 경로: {data_folder}")

    files = [os.path.join(root, file)
             for root, _, files in os.walk(data_folder)
             for file in files if file.endswith('.json')]

    print(f"발견된 파일들: {len(files)}개")

    # 병렬로 파일 로드
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(load_single_file, files))

    # None이 아닌 결과만 필터링
    qa_pairs = [result for result in results if result is not None]

    print(f"로드된 QA 쌍 개수: {len(qa_pairs)}")
    return qa_pairs

# 6. 캐시를 사용한 데이터 로드 함수
def load_data_with_cache(data_folder, cache_file):
    """캐시를 사용해 데이터를 로드하는 함수"""
    # 캐시된 데이터가 있으면 로드
    data = load_data_from_cache(cache_file)
    if data is not None:
        return data

    # 없으면 데이터 병렬 로드 후 캐시 저장
    data = load_data_parallel(data_folder)
    save_data_to_cache(data, cache_file)
    return data

# 7. 데이터 경로 설정
train_extracted_path = './extracted_files/Training/unzipped'
val_extracted_path = './extracted_files/Validation/unzipped'

# 캐시된 데이터를 로드
train_qa_pairs = load_data_with_cache(train_extracted_path, 'train_data_cache.pkl')
val_qa_pairs = load_data_with_cache(val_extracted_path, 'val_data_cache.pkl')

# 데이터가 비어 있는지 확인
print(f"Training 데이터 쌍 개수: {len(train_qa_pairs)}")
print(f"Validation 데이터 쌍 개수: {len(val_qa_pairs)}")

if len(train_qa_pairs) == 0 or len(val_qa_pairs) == 0:
    raise ValueError("데이터셋이 비어 있습니다. 데이터 로드 또는 전처리를 확인하세요.")

# 8. BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")

# 9. Dataset 및 DataLoader 설정
class QADataset(Dataset):
    def __init__(self, qa_pairs, tokenizer, max_len=512):
        self.qa_pairs = qa_pairs
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        question, context, answer, answer_start = self.qa_pairs[idx]

        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        answer_end = answer_start + len(self.tokenizer.encode(answer, add_special_tokens=False))

        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        token_type_ids = inputs["token_type_ids"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            "start_positions": torch.tensor(answer_start),
            "end_positions": torch.tensor(answer_end)
        }

train_dataset = QADataset(train_qa_pairs, tokenizer)
val_dataset = QADataset(val_qa_pairs, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# 10. 모델 학습 및 검증
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            start_positions = batch["start_positions"].to(device)
            end_positions = batch["end_positions"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, start_positions=start_positions, end_positions=end_positions)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(data_loader)

# 11. 학습 루프
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = evaluate(model, val_loader, device)
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

# 12. Gradio 웹 인터페이스 구현
import gradio as gr

def predict_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    start = torch.argmax(start_scores)
    end = torch.argmax(end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start:end]))
    return answer

def chatbot_interface(user_input):
    context = "국회 회의록에서의 발언 내용"  # 실제 문맥으로 대체해야 함
    answer = predict_answer(user_input, context)
    return answer

# Gradio 웹 인터페이스 실행
interface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="BERT 기반 국회 회의록 챗봇")
interface.launch()


데이터 폴더 경로: ./extracted_files/Training/unzipped
발견된 파일들: 35233개
로드된 QA 쌍 개수: 8594
데이터를 캐시에 저장했습니다: train_data_cache.pkl
데이터 폴더 경로: ./extracted_files/Validation/unzipped
발견된 파일들: 4400개
로드된 QA 쌍 개수: 1081
데이터를 캐시에 저장했습니다: val_data_cache.pkl
Training 데이터 쌍 개수: 8594
Validation 데이터 쌍 개수: 1081


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/user/miniforge3/envs/dml/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Token indices sequence length is longer than the specified maximum sequence length for this model (2216 > 512). Running this sequence through the m

Epoch 1/3


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
